In [ ]:
! pip install environs facebook_business cyksuid toolz psycopg2-binary typing_json backoff

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
from environs import Env


env = Env()
env.read_env('.env-vital')
from marketing import Marketing
m = Marketing(env)

In [ ]:

from malaria import update_ads

update_ads()

In [75]:
adsets = m.account.get_ad_sets(fields=['lifetime_budget', 'instagram_actor_id', 'campaign_id', 'targeting'])

In [78]:
m.account.create_ad_label

<bound method AdAccount.create_ad_label of <AdAccount> {
    "id": "act_2240415876083895"
}>

In [34]:
from malaria import load_creatives

c = load_creatives('config/creatives.json', 'hindi')

cc = c.creatives[0]

In [42]:
m.account.get_ad_labels()

[<AdLabel> {
    "id": "23845036050710518",
    "name": "vlab"
}]

In [ ]:
m.account.get_ad_creatives_by_labels(params={'ad_label_ids': ['23845036050710518']})

In [64]:

# its idempotent!!!
m.account.create_ad_label(params={'name': 'vlab'})

<AdLabel> {
    "id": "23845036050710518"
}

In [4]:
import pandas as pd
cities = pd.read_csv('output/cities.csv')
cities = cities[cities.rad >= 1.0]

locs = [(r.lat, r.lng, r.rad) for _, r 
        in cities[:200].iterrows()]

cities.distname.unique().shape

(142,)

In [ ]:
import re
from facebook_business.adobjects.adcreative import AdCreative

def delete_our_creatives(account):
    # TODO: change for get by label!
    ids = [a['id'] for a in account.get_ad_creatives(fields=['name']) 
           if re.match('vlab-', a['name'])]

    print(len(ids))

    i = 0
    for id_ in ids:
        try:
            c = AdCreative(id_)
            c.api_delete()
            i += 1
            print(i)
        except:
            print('nope!')



# delete_our_creatives(m.account)

In [155]:
import pandas as pd

def clean_districts(df):
    dist_var = 'Districtname'
    state_var = 'statename'

    states = ['Jharkhand', 'Chhatisgarh', 'Orissa', 'Odisha', 'Uttar Pradesh']
    states = [s.upper() for s in states]
 
    df = df[~df[dist_var].isna()].reset_index(drop=True)
    df = df[df[state_var].isin(states)].reset_index(drop=True)


    
    df[dist_var] = df[dist_var].map(lambda x: x.upper())
    
    renames = {
        'PRAYAGRAJ ALLAHABAD': 'ALLAHABAD',
        'HAZARIBAGH': 'HAZARIBAG'
    }

    for old, new_ in renames.items():
        df = df.where(df[dist_var] != old, new_)

    blacklist = ['AURANGABAD', 'JHAJJAR', 'SHOPIAN', 'RAIGARH(MH)', 'HARIDWAR']
    df = df[~df[dist_var].isin(blacklist)]
    return df

df = pd.read_csv('./India_MNM/Geography/PINcodes/PINcode_2015.csv', engine='python')

df = clean_districts(df)

In [55]:
cities['District'] = cities.distname.str.upper()

In [127]:
dist_var = 'Districtname'

In [ ]:
cities[~cities.District.isin(df[dist_var].unique())].District.unique()

In [ ]:
df.District.unique()

In [ ]:
dd = cities.merge(df, on='District', how='left')

dd[dd['Division Name'].isna()].District.unique()

In [ ]:
df.District.unique()

In [ ]:
def foo(df):
    return df[dist_var].unique().shape[0] > 1

df.groupby('pincode').filter(foo).reset_index(drop=True)[['pincode', dist_var]].sort_values('pincode')[60:100]

In [ ]:
df

In [ ]:
districts = 